In [1]:
!pip install pandas ultralytics tensorflow

In [2]:
import os
HOME = os.getcwd()
print(HOME)

f:\Github\KU\Driver-drowsiness-detection\tranining_model_manual


In [3]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.189  Python-3.11.13 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
Setup complete  (16 CPUs, 31.8 GB RAM, 11.6/232.9 GB disk)


### Traning model

In [ ]:
from ultralytics import YOLO

# โหลดโมเดล YOLOv9 (เลือกขนาด: yolov9n, yolov9s, yolov9m, yolov9l, yolov9e)
# โหลดโมเดล YOLOv8 (เลือกขนาด: yolov8n)
model = YOLO("yolo11m.pt") # c = compact, e = extra large

# เทรน
model.train(
    data="pool_data_set/data_set_1/data.yaml",   # path ไปที่ไฟล์ yaml
    epochs=50,                   # จำนวนรอบการเทรน
    imgsz=640,                   # ขนาดภาพ
    batch=32,                    # batch size
    workers=3,                   # worker สำหรับโหลดข้อมูล
    device=0,                     # ใช้ GPU (0) ถ้าไม่มีให้ใส่ 'cpu'
    project="model",
    name="model"
)


New https://pypi.org/project/ultralytics/8.3.202 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.189  Python-3.11.13 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=model9_yolov11n_v1i2, nbs=64, nms=False, opset=None, opt

RuntimeError: Dataset '../data/data.yaml' error  '../data/data.yaml' does not exist

# Convert model for TFlite for use in Android

In [9]:
from ultralytics import YOLO

# Load your custom trained model
model = YOLO("model/model8_yolov11n_v1i_1/weights/best.pt")

# Export to TFLite format
model.export(format="tflite", imgsz=640)



Ultralytics 8.3.202 🚀 Python-3.10.18 torch-2.8.0 CPU (Apple M4)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'model/model8_yolov11n_v1i_1/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.2 MB)

TensorFlow SavedModel: starting export with tensorflow 2.19.0...

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.68...
ONNX: export success ✅ 1.1s, saved as 'model/model8_yolov11n_v1i_1/weights/best.onnx' (10.2 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.28.2...
Saved artifact at 'model/model8_yolov11n_v1i_1/weights/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 640, 640, 3), dtype=tf.float32, name='images')
Output Type:
  TensorSpec(shape=(1, 5, 8400), dtype=tf.float32, name=None)
Captures:
  4420433728: TensorSpec(shape=(4, 2), dtype=tf.int32,

I0000 00:00:1758211111.003499   57333 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1758211111.003562   57333 single_machine.cc:374] Starting new session
W0000 00:00:1758211111.709325   57333 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1758211111.709336   57333 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1758211112.084911   57333 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1758211112.084971   57333 single_machine.cc:374] Starting new session


TensorFlow SavedModel: export success ✅ 7.5s, saved as 'model/model8_yolov11n_v1i_1/weights/best_saved_model' (25.6 MB)

TensorFlow Lite: starting export with tensorflow 2.19.0...
TensorFlow Lite: export success ✅ 0.0s, saved as 'model/model8_yolov11n_v1i_1/weights/best_saved_model/best_float32.tflite' (10.1 MB)

Export complete (7.6s)
Results saved to /Users/tutamini/Documents/Github/TrainPotHole/model/model8_yolov11n_v1i_1/weights
Predict:         yolo predict task=detect model=model/model8_yolov11n_v1i_1/weights/best_saved_model/best_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=model/model8_yolov11n_v1i_1/weights/best_saved_model/best_float32.tflite imgsz=640 data=pothole.v1i.yolov8/data.yaml  
Visualize:       https://netron.app


W0000 00:00:1758211112.843207   57333 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1758211112.843217   57333 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


'model/model8_yolov11n_v1i_1/weights/best_saved_model/best_float32.tflite'

In [ ]:
import tensorflow as tf

# โหลดโมเดล SavedModel
converter = tf.lite.TFLiteConverter.from_saved_model("model/model1/weights/best.pt")

# (optional) ลดขนาดโมเดล
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# แปลงเป็น TFLite
tflite_model = converter.convert()

# บันทึกไฟล์
with open("model1.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ แปลงเสร็จแล้ว: model1.tflite")
